In [2]:
import csv
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pnd
import seaborn as sns
import sklearn
import timeit


from collections import Counter
from functions.KNN import*
from functions.PCA import pca
from pathlib import Path
from scipy import spatial
from scipy.spatial import KDTree
from scipy.stats import mode
from sklearn.metrics import confusion_matrix


dataset = 1
match dataset:
    case 1:
        test = 'fashion-mnist_test.csv'
        train = 'fashion-mnist_train.csv'
    case 2: 
        test = "mnist_test.csv"
        train = "mnist_train.csv"
testdata = pnd.read_csv(test)
traindata = pnd.read_csv(train)


testdata_pixel = testdata.drop(testdata.columns[0], axis=1).to_numpy()
traindata_pixel = traindata.drop(traindata.columns[0], axis=1).to_numpy()
label_train = traindata[traindata.columns[0]].to_numpy()
label_test = testdata[testdata.columns[0]].to_numpy()

class_names = np.array(["T-shirt / Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"])

In [ ]:
filepath = Path('C:/Users/awark')  
filepath.parent.mkdir(parents=True, exist_ok=True)  

PCs_train, PCs_test = pca(traindata_pixel,testdata_pixel,1)

def knn_kdtree(PCs_train, PCs_test, k, label_train, label_test, testsize):
    """returns the accuray of the KNN for test images

    Args:
        PCs_train (numpy array): transformed training data
        PCs_test (numpy array): transformed testing data
        k (int): number of neighbours 
        label_train (numpy array 1D): labels of training data
        label_test (numpy array 1D): labels of testing data
        testsize (int): number of testing images you want to classify at a time
        
    Returns:
        knn for number of testsize images
    """
    result = np.array([])
    kd_tree = spatial.KDTree(PCs_train,leafsize=10)
    for i in range(0, len(label_test), testsize):
        dist, neighbour_index = kd_tree.query(PCs_test[i:i+testsize, None],p=2,k=k, workers = -1)
        neighbour_label = label_train[neighbour_index]
        batch_result = [mode(neighbour_label,axis=2)[0]]
        batch_result = np.array(batch_result)
        batch_result = np.squeeze(batch_result)
        batch_result = batch_result.astype(int)
        result = result.astype(int)
        result = np.concatenate((result, batch_result), axis=0)
    return result

In [ ]:
total_var_results=[]
for k in range(2, 16): 
    new_column = knn_kdtree(PCs_train, PCs_test, k, label_train, label_test, 40).tolist()
    total_var_results.append(new_column)
total_var_results = pnd.DataFrame(total_var_results).transpose()  
total_var_results.to_csv('out.csv', sep=',', index=False) 